In [26]:
import requests
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="api.env")

api_key = os.getenv("API_KEY_LASTFM") 

In [27]:
import json
import pandas as pd

In [32]:
import csv
with open('list_artist_rock.csv', "r") as archivo:
    lector = csv.reader(archivo)
    lista = list(lector) 

In [28]:
def stats(id_genero, artist, api_key):
    url = 'http://ws.audioscrobbler.com/2.0/'
    params = {
        'method': 'artist.getinfo',
        'artist' : artist,
        'api_key':  api_key,
        'format': 'json',
        'lang': 'es'
        }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Da error")
    else:
        datos = response.json()
        info = datos['artist']['stats'] #extraer listeners y plays por separado
        info_artista = {
            'id_genero': id_genero,
            'artista': artist, 
            'oyentes': info.get('listeners'),
            'reproducciones': info.get('playcount'),
            }
        
    return info_artista

In [29]:
generos = {
    'classical': 1,
    'jazz': 2,
    'pop': 3,
    'rock': 4
}

In [31]:
for i in lista:
    print(i)

NameError: name 'lista' is not defined

In [33]:
lista = [i[0] if isinstance(i, list) else i for i in lista]

In [ ]:
statistics = []
limit = 200
for contador, nombre_artista in enumerate(lista):
    if contador >= limit:
        break
    statistics.append(stats(nombre_artista, api_key))

In [ ]:
statistics

In [ ]:
df_rock = pd.DataFrame(statistics)

In [ ]:
df_rock.head()

-----------------

In [34]:
df_classical = pd.read_csv("list_artist_classical.csv")
df_jazz = pd.read_csv("list_artist_jazz.csv")
df_pop = pd.read_csv("list_artist_pop.csv")
df_rock = pd.read_csv("list_artist_rock.csv")

In [35]:
csvs_generos = {
    "list_artist_classical.csv": 1,
    "list_artist_jazz.csv": 2,
    "list_artist_pop.csv": 3,
    "list_artist_rock.csv": 4
}

In [36]:
statistics = []
limit = 200

for archivo_csv, id_genero in csvs_generos.items():
    df = pd.read_csv(archivo_csv, header=None, names=['nombre_artista'])

    artistas = df['nombre_artista']

    for contador, nombre_artista in enumerate(artistas):
        if contador >= limit:
            break

        try:
            data = stats(id_genero, nombre_artista, api_key)
            if data:
                statistics.append(data)
        except Exception as e:
            print(f"Error con artista '{nombre_artista}': {e}")

In [37]:
import mysql.connector
from mysql.connector import errorcode

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='Musicproject')

print(cnx)


In [ ]:
mycursor = cnx.cursor()

query = "INSERT INTO Tabla_stats (id_genero, nombre_artista, oyentes, reproduccion) VALUES (%s, %s, %s, %s)"

# Convertir lista de diccionarios a lista de tuplas
valores = [(d['id_genero'], d['artista'], d['oyentes'], d['reproducciones']) for d in statistics]

try:
    mycursor.executemany(query, valores)
    cnx.commit()
    print(mycursor.rowcount, "registros insertados.")
except mysql.connector.Error as err:
    print(err)

In [ ]:
cnx.close()